## 載入套件

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## 取得網頁內容

In [2]:
resp = requests.get('https://www.investing.com/indices/hsi-volatility')
soup = BeautifulSoup(resp.text, 'html.parser')
soup


<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html>
<head>
<title>403 ERR#1: You are banned from this site.  Please contact via a different client configuration if you believe that this is a mistake.</title>
</head>
<body>
<h1>Error 403 ERR#1: You are banned from this site.  Please contact via a different client configuration if you believe that this is a mistake.</h1>
<p>ERR#1: You are banned from this site.  Please contact via a different client configuration if you believe that this is a mistake.</p>
<h3>Guru Meditation:</h3>
<p>XID: 699116105</p>
<hr/>
<p>Varnish cache server</p>
</body>
</html>

In [26]:
# 上面的方法會被阻擋
# 因此加上 headers 讓對方認為我們是透過瀏覽器逛網站的使用者
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
resp = requests.get('https://www.investing.com/indices/hsi-volatility-historical-data', headers=headers)
soup = BeautifulSoup(resp.text, 'html.parser')
# soup
str(soup)[:3000] # 把結果放到 github 會變太長，所以只顯示前面的部分

'<!DOCTYPE HTML>\n\n<html class="com" dir="ltr" geo="TW" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:schema="http://schema.org/">\n<head>\n<script>\n        (function(i,s,o,g,r,a,m){i[\'GoogleAnalyticsObject\']=r;i[r]=i[r]||function(){\n            (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n            m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n        })(window,document,\'script\',\'//www.google-analytics.com/analytics.js\',\'ga\');\n    </script>\n<link href="https://i-invdn-com.akamaized.net" rel="dns-prefetch"/>\n<link href="https://a-invdn-com.akamaized.net" rel="dns-prefetch"/>\n<title>HSI Volatility Historical Rates - Investing.com</title>\n<link href="https://www.investing.com/indices/hsi-volatility-historical-data" hreflang="x-default" rel="alternate"/>\n<link href="https://il.investing.com/indices/hsi-volatility-historical-data" hreflang="he-il" rel="alternate"/>\n<link href="htt

In [23]:
# 取得目前報價

# 兩種選取 element 的方式
# 方式一
last_time = soup.select('.pid-49577-time')[0].text
last_price = soup.select("#last_last")[0].text

# 方式二
# last_time = soup.find_all('span', class_='pid-49577-time')[0].text
# last_price = soup.find(id='last_last').text

print('last_time = ', last_time)
print('last_price = ', last_price)

last_time =  22:25:30
last_price =  16.89


In [28]:
# 將上面程式整理成函示
def getLastPrice():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
    resp = requests.get('https://www.investing.com/indices/hsi-volatility-historical-data', headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    last_time = soup.select('.pid-49577-time')[0].text
    last_price = soup.select("#last_last")[0].text
    print('last_time = {}, last_price = {}'.format(last_time, last_price))

In [29]:
getLastPrice()

last_time = 22:52:30, last_price = 16.84


## 設定時間執行程式
schedule 介紹: [https://zhuanlan.zhihu.com/p/23086148](https://zhuanlan.zhihu.com/p/23086148)

要停止執行的時候，請點擊選單 Run 右邊的方塊

In [32]:
import schedule
import time

def job():
    getLastPrice()  # 執行上面整理的函示

job()  # 先執行第一次
# schedule.every(3).seconds.do(job)  # 幾秒鐘執行一次
schedule.every(1).minutes.do(job)  # 幾分鐘執行一次

while True:
    schedule.run_pending()
    time.sleep(1)

last_time = 22:52:30, last_price = 16.84
last_time = 22:52:30, last_price = 16.84
last_time = 22:54:30, last_price = 16.85
last_time = 22:54:30, last_price = 16.85
last_time = 22:56:30, last_price = 16.88
last_time = 22:57:30, last_price = 16.81
last_time = 22:58:30, last_price = 16.90
last_time = 22:59:15, last_price = 16.91


KeyboardInterrupt: 

In [ ]:
# 我發現再次執行 schedule 的時候
# 之前的排程會被記錄著
# 用 clear 把它清掉
schedule.clear()

## 取得 Streaming Chart 資料

In [33]:
# t: time
# o: open
# h: high
# l: low
# c: close

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
resp = requests.get('https://tvc4.forexpros.com/eaa0af731455d8bbc48a5b286ca79c06/1553505233/1/1/8/history?symbol=49577&resolution=D&from=1522401241&to=1553505301', headers=headers)
df = pd.DataFrame(resp.json())
df.head()

,t,c,o,h,l,v,vo,s
0,1522713600,24.969999,27.750000,27.950001,24.59,n/a,n/a,ok
1,1522800000,25.900000,24.420000,26.020000,23.34,n/a,n/a,ok
2,1522972800,24.590000,23.420000,25.559999,23.33,n/a,n/a,ok
3,1523232000,23.120001,23.450001,23.980000,17.85,n/a,n/a,ok
4,1523318400,22.309999,22.740000,22.990000,21.27,n/a,n/a,ok


In [34]:
# 轉換 t 的時間格式
df['t'] = df['t'].apply(lambda x: time.strftime("%Y-%m-%d", time.localtime(x)))
df.head()

,t,c,o,h,l,v,vo,s
0,2018-04-03,24.969999,27.750000,27.950001,24.59,n/a,n/a,ok
1,2018-04-04,25.900000,24.420000,26.020000,23.34,n/a,n/a,ok
2,2018-04-06,24.590000,23.420000,25.559999,23.33,n/a,n/a,ok
3,2018-04-09,23.120001,23.450001,23.980000,17.85,n/a,n/a,ok
4,2018-04-10,22.309999,22.740000,22.990000,21.27,n/a,n/a,ok


In [35]:
# 取得最後一筆 close 資料
df['c'].iloc[-1]

18.139999389648

## 網址內的神秘參數

```
在網址裡面有一段參數 from=1522401241&to=1553505301
感覺應該是時間區間
```

In [36]:
import time
 
ticks = time.time()
print("當前時間:", ticks)

當前時間: 1553570239.524992


In [37]:
timestamp = 1553524786.706065

# 轉換成新的時間格式
dt = time.strftime("%Y-%m-%d", time.localtime(timestamp))
# dt = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))

dt

'2019-03-25'

In [38]:
date = "2019-01-31"
ticks = time.mktime(time.strptime(date, "%Y-%m-%d"))
ticks

1548864000.0

In [39]:
# 整理成函數方便取得時間
def getTime(timestamp):
    # 轉換成新的時間格式
    dt = time.strftime("%Y-%m-%d", time.localtime(timestamp))
    print(dt)

In [40]:
getTime(1522401241)
getTime(1553505301)

2018-03-30
2019-03-25


In [41]:
# 整理成函數方便取得時間 ticks
def getTicks(date):
    ticks = time.mktime(time.strptime(date, "%Y-%m-%d"))
    print(ticks)

In [42]:
getTicks('2018-03-30')
getTicks('2019-03-25')

1522339200.0
1553443200.0


## 使用 csv 儲存資料

In [43]:
# 存檔
# index: 設定是否加上索引
# encoding: 設定編碼格式
df.to_csv('vhsi_price.csv', index=False, encoding='utf_8_sig')

In [44]:
# 讀檔
# index_col: 讀取時，把某行當成索引
df_csv = pd.read_csv('vhsi_price.csv', index_col=['t'])
df_csv.head()

,c,o,h,l,v,vo,s
t,,,,,,,
2018-04-03,24.969999,27.750000,27.950001,24.59,NaN,NaN,ok
2018-04-04,25.900000,24.420000,26.020000,23.34,NaN,NaN,ok
2018-04-06,24.590000,23.420000,25.559999,23.33,NaN,NaN,ok
2018-04-09,23.120001,23.450001,23.980000,17.85,NaN,NaN,ok
2018-04-10,22.309999,22.740000,22.990000,21.27,NaN,NaN,ok


## 使用 sqlite3 儲存資料
DB Browser for SQLite: 
[https://sqlitebrowser.org/](https://sqlitebrowser.org/)

In [45]:
import sqlite3
conn = sqlite3.connect('vhsi_price.sqlite3')

# 存檔
# index: 設定是否加上索引
# if_exists='replace'，假如 sql 中已經有 daily_price 這個 dataframe，則直接覆蓋掉它
df.to_sql('vhsi_price', conn, index=False, if_exists='replace')

In [46]:
# 讀檔
# index_col: 讀取時，把某行當成索引
df_sqlite = pd.read_sql('select * from vhsi_price', conn, index_col=['t'])
df_sqlite.head()

,c,o,h,l,v,vo,s
t,,,,,,,
2018-04-03,24.969999,27.750000,27.950001,24.59,n/a,n/a,ok
2018-04-04,25.900000,24.420000,26.020000,23.34,n/a,n/a,ok
2018-04-06,24.590000,23.420000,25.559999,23.33,n/a,n/a,ok
2018-04-09,23.120001,23.450001,23.980000,17.85,n/a,n/a,ok
2018-04-10,22.309999,22.740000,22.990000,21.27,n/a,n/a,ok
